## Imports

In [1]:
# necessary imports
import cv2
from pathlib import Path
import glob
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage import io
import tensorflow as tf
import numpy as np
import matplotlib
import cv2
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image as kimg

## Model Architechture

In [4]:
# defining input shape of the model
shape=(200,200,3)

In [6]:
def model():
    
    '''
    This function contains deep learning model consisting of CNN and LSTM layers. CNN is used for extracting useful
    features from the image pixels and LSTM is used to remember sequence of frames.
    input shape : (number of frames, image width, image height, channel) - eg. (12,200,200,3)
    output : number of classes - (no of exercise that we wish to classify)
    checkpoint : relative path of checkpoint file (.pkl)
    '''

    model = Sequential()
    model.add(Conv2D(32, (5,5), padding='same', activation='relu',input_shape=(200, 200, 3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(32, (5,5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (5,5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (5,5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (5,5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (5,5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))

    model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    model.load_weights('cp-050.pkl')
    
    return model

In [7]:
#calling the model
model_exercise = model()

## Save Model as pb

In [9]:
filepath = "./PretikaFinalModel/"
model_exercise.save(filepath)

INFO:tensorflow:Assets written to: ./PretikaFinalModel/assets


## Tflite converter

In [10]:
converter = tf.lite.TFLiteConverter.from_saved_model(filepath)

In [11]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
# converter.allow_custom_ops=True

In [12]:
#convert into tflite and save to variable tflite_model
tflite_model = converter.convert()

In [13]:
#create an empty .tflite file and write the converted model in it
converted_path = './PretikaFinalModel/Jun20.tflite'
open(converted_path, "wb").write(tflite_model)

2223632

In [14]:
tflite_model = './PretikaFinalModel/Jun20.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model)
interpreter.allocate_tensors()
#get input and output tensors
input_details = interpreter.get_input_details()
print("input_shape:", input_details[0]['shape'])
output_details = interpreter.get_output_details()
print("output_shape:", output_details[0]['shape'])

input_shape: [  1 200 200   3]
output_shape: [1 3]


In [41]:
img = kimg.load_img('mild11.jpg',target_size=(200,200))
images = kimg.img_to_array(img)
print(images.shape)
images = np.expand_dims(images, axis=0) #for matching input shape of the model
images_NP = np.array(images,dtype=np.float32)
input_index = interpreter.get_input_details()[0]["index"]
print("input_index : ",input_index)
interpreter.set_tensor(input_index,images_NP)
interpreter.invoke()
prediction = interpreter.get_tensor(output_details[0]['index'])
print(prediction)

(200, 200, 3)
input_index :  0
[[1. 0. 0.]]
